# 4: Azure OpenAI Function Calling Feature

## Setup

In [1]:
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

-------------------------

## 1. Using an illustrative example

In [2]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location. 
    The default unit when not specified is fahrenheit"""
    if "new york" in location.lower():
        return json.dumps(
            {"location": "New York", "temperature": "40", "unit": unit}
        )
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "50", "unit": unit}
        )
    elif "las vegas" in location.lower():
        return json.dumps(
            {"location": "Las Vegas", "temperature": "70", "unit": unit}
        )
    else:
        return json.dumps(
            {"location": location, "temperature": "unknown"}
        )

get_current_weather("New York")

'{"location": "New York", "temperature": "40", "unit": "fahrenheit"}'

## Define the tools

In [3]:
messages = [
    {"role": "user",
    "content": """What's the weather like in San Francisco,
                New York, and Las Vegass?"""
    }
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": """Get the current weather in a given
                            location.The default unit when not
                            specified is fahrenheit""",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": """The city and state,
                                        e.g. San Francisco, CA""",
                    },
                    "unit": {
                        "type": "string",
                        "default":"fahrenheit",
                        "enum": [ "fahrenheit", "celsius"],
                        "description": """The messuring unit for
                                        the temperature.
                                        If not explicitly specified
                                        the default unit is 
                                        fahrenheit"""
                    },
                },
                "required": ["location"],
            },
        },
    }
]

## Use the function calling

In [4]:
response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_API_MODEL"),
    messages=messages,
    tools=tools,
    tool_choice="auto", 
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    print (tool_calls)
    
    available_functions = {
        "get_current_weather": get_current_weather,
    } 
    messages.append(response_message)  
    
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  
    print (messages)

[ChatCompletionMessageToolCall(id='call_QvVb8fEUGx7BnI4mP8y1JNvI', function=Function(arguments='{"location": "San Francisco, CA"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_bXOBMkulllvjks52C3yK5oY3', function=Function(arguments='{"location": "New York, NY"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_Rh9FJ2tcDbrBJOIk3zRPJngj', function=Function(arguments='{"location": "Las Vegas, NV"}', name='get_current_weather'), type='function')]
[{'role': 'user', 'content': "What's the weather like in San Francisco,\n                New York, and Las Vegass?"}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_QvVb8fEUGx7BnI4mP8y1JNvI', function=Function(arguments='{"location": "San Francisco, CA"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_bXOBMkulllvjks52C3yK5oY3', function=Function(argument

-----------------

## 2. Using our SQL database

In [5]:
from sqlalchemy import create_engine
import pandas as pd

df = pd.read_csv("./data/all-states-history.csv").fillna(value = 0)

In [6]:
database_file_path = "./db/test.db"

engine = create_engine(f'sqlite:///{database_file_path}')

df.to_sql(
    'all_states_history',
    con=engine,
    if_exists='replace',
    index=False)

20780

## Create two functions

In [7]:
import numpy as np
from sqlalchemy import text

def get_hospitalized_increase_for_state_on_date(state_abbr, specific_date):
    try:
        query = f"""
        SELECT date, hospitalizedIncrease
        FROM all_states_history
        WHERE state = '{state_abbr}' AND date = '{specific_date}';
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        if not result.empty:
            return result.to_dict('records')[0]
        else:
            return np.nan
    except Exception as e:
        print(e)
        return np.nan

In [8]:
def get_positive_cases_for_state_on_date(state_abbr, specific_date):
    try:
        query = f"""
        SELECT date, state, positiveIncrease AS positive_cases
        FROM all_states_history
        WHERE state = '{state_abbr}' AND date = '{specific_date}';
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        if not result.empty:
            return result.to_dict('records')[0]
        else:
            return np.nan
    except Exception as e:
        print(e)
        return np.nan

In [9]:
get_hospitalized_increase_for_state_on_date("AK","2021-03-05")

{'date': '2021-03-05', 'hospitalizedIncrease': 3}

## Execute the function calling against the SQL database

In [10]:
messages = [
    {"role": "user",
    "content": """ how many hospitalized people we had in Alaska
                    the 2021-03-05?"""
    }
]

In [11]:
tools_sql = [
    {
        "type": "function",
        "function": {
            "name": "get_hospitalized_increase_for_state_on_date",
            "description": """Retrieves the daily increase in
                              hospitalizations for a specific state
                              on a specific date.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "state_abbr": {
                        "type": "string",
                        "description": """The abbreviation of the state
                                          (e.g., 'NY', 'CA')."""
                    },
                    "specific_date": {
                        "type": "string",
                        "description": """The specific date for
                                          the query in 'YYYY-MM-DD'
                                          format."""
                    }
                },
                "required": ["state_abbr", "specific_date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_positive_cases_for_state_on_date",
            "description": """Retrieves the daily increase in 
                              positive cases for a specific state
                              on a specific date.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "state_abbr": {
                        "type": "string",
                        "description": """The abbreviation of the 
                                          state (e.g., 'NY', 'CA')."""
                    },
                    "specific_date": {
                        "type": "string",
                        "description": """The specific date for the
                                          query in 'YYYY-MM-DD'
                                          format."""
                    }
                },
                "required": ["state_abbr", "specific_date"]
            }
        }
    }
]

In [12]:
response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_API_MODEL"),
    messages=messages,
    tools=tools_sql,
    tool_choice="auto",
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    print (tool_calls)
    
    available_functions = {
        "get_positive_cases_for_state_on_date": get_positive_cases_for_state_on_date,
        "get_hospitalized_increase_for_state_on_date":get_hospitalized_increase_for_state_on_date
    }  
    messages.append(response_message)  
   
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            state_abbr=function_args.get("state_abbr"),
            specific_date=function_args.get("specific_date"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": str(function_response),
            }
        ) 
    print(messages)

[ChatCompletionMessageToolCall(id='call_NtqjlRA6Taj7mkbSoJcMLoOr', function=Function(arguments='{"state_abbr":"AK","specific_date":"2021-03-05"}', name='get_hospitalized_increase_for_state_on_date'), type='function')]
[{'role': 'user', 'content': ' how many hospitalized people we had in Alaska\n                    the 2021-03-05?'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_NtqjlRA6Taj7mkbSoJcMLoOr', function=Function(arguments='{"state_abbr":"AK","specific_date":"2021-03-05"}', name='get_hospitalized_increase_for_state_on_date'), type='function')]), {'tool_call_id': 'call_NtqjlRA6Taj7mkbSoJcMLoOr', 'role': 'tool', 'name': 'get_hospitalized_increase_for_state_on_date', 'content': "{'date': '2021-03-05', 'hospitalizedIncrease': 3}"}]


In [13]:
second_response = client.chat.completions.create(
            model=os.getenv("AZURE_OPENAI_API_MODEL"),
            messages=messages,
        )
print (second_response)

ChatCompletion(id='chatcmpl-9ak0KIzwwgPyWH9n7l7enx7ZflIAU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='On March 5, 2021, there was an increase of 3 individuals hospitalized in Alaska.', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1718543676, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_abc28019ad', usage=CompletionUsage(completion_tokens=21, prompt_tokens=95, total_tokens=116), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}

In [14]:
second_response.choices[0].message.content

'On March 5, 2021, there was an increase of 3 individuals hospitalized in Alaska.'